In [1]:
from models.arcs import load_alt_model_a, load_inf_model, GenPhiloText
from utilities.loaders import load_file
from utilities.preprocessors import preprocess, map_value_to_index, init_sequences_a, init_sequences_b, decode_predictions
from utilities.visualizers import export_results

from tensorflow.keras.losses import CategoricalCrossentropy as cce_loss
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import CategoricalAccuracy, CategoricalCrossentropy as cce_metric

from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model
import tensorflow as tf

%load_ext autoreload
%autoreload 2

In [2]:
corpus = load_file('./data/notes.txt')

In [3]:
corpus[:500]

'A simple idea “What is the meaning of life?”\nI asked as I learned through the works of Camus? \nOne step down, I felt a yearning of meaning in this world.\nIn this yearning I stumbled upon eastern philosophy;\nIkigai as the Japanese philosophers called it was a considerable way for me to find meaning at that certain point in my life. \nFollowed then another idea, a leap of faith as Kierkegaard would call it, yet I had no idea this was his idea. \nCalm followed after the storm, and then I took another'

In [4]:
len(corpus)

226750

In [5]:
chars = sorted(list(set(corpus)))
chars

['\n',
 ' ',
 '!',
 '"',
 '&',
 "'",
 '(',
 ')',
 ',',
 '-',
 '.',
 '/',
 '0',
 '1',
 '2',
 '3',
 '4',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'Y',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '´',
 'ç',
 'é',
 'ï',
 '–',
 '—',
 '‘',
 '’',
 '“',
 '”',
 '…']

# Preprocessing corpus
* replace quotation marks like this '“'/'”' with this instead '"'
* replace single quotation marks like this '‘'/'’' with ''' instead
* replace this hyphen '–' with this hyphen '—'
* lowercase all words (for now)
* replace 3 consecutive '.' with  '…' instead

In [6]:
corpus = preprocess(corpus)
corpus[:500]

'a simple idea "what is the meaning of life?"\ni asked as i learned through the works of camus? \none step down, i felt a yearning of meaning in this world.\nin this yearning i stumbled upon eastern philosophy;\nikigai as the japanese philosophers called it was a considerable way for me to find meaning at that certain point in my life. \nfollowed then another idea, a leap of faith as kierkegaard would call it, yet i had no idea this was his idea. \ncalm followed after the storm, and then i took another'

In [7]:
len(corpus)

226961

In [8]:
chars = sorted(list(set(corpus)))
# chars = ['[UNK]'] + chars
chars

['\n',
 ' ',
 '!',
 '"',
 '&',
 "'",
 '(',
 ')',
 ',',
 '-',
 '.',
 '/',
 '0',
 '1',
 '2',
 '3',
 '4',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '?',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '´',
 'ç',
 'é',
 'ï',
 '—',
 '…']

# Creating mapper from a unique character to its respective index

In [9]:
char_to_idx = map_value_to_index(chars)
idx_to_char = map_value_to_index(chars, inverted=True)

In [10]:
char_to_idx

In [11]:
idx_to_char

In [12]:
n_unique = len(char_to_idx.get_vocabulary())
n_unique

57

In [13]:
n_time_steps = 100
X, Y = init_sequences_a(corpus, char_to_idx, T_x=n_time_steps)
X

<tf.Tensor: shape=(226861, 100), dtype=int64, numpy=
array([[25,  2, 43, ..., 29,  2, 43],
       [ 2, 43, 33, ...,  2, 43, 44],
       [43, 33, 37, ..., 43, 44, 29],
       ...,
       [39, 42,  2, ..., 29, 36, 33],
       [42,  2, 44, ..., 36, 33, 29],
       [ 2, 44, 42, ..., 33, 29, 30]], dtype=int64)>

In [14]:
X.shape

TensorShape([226861, 100])

In [15]:
Y

<tf.Tensor: shape=(226861,), dtype=int64, numpy=array([44, 29, 40, ..., 29, 30, 11], dtype=int64)>

# convert Y data's indeces to their one hot vector representation

In [16]:
Y = tf.one_hot(Y, depth=n_unique)
Y

<tf.Tensor: shape=(226861, 57), dtype=float32, numpy=
array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)>

In [17]:
Y[3]

<tf.Tensor: shape=(57,), dtype=float32, numpy=
array([0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0.], dtype=float32)>

In [18]:
# number of examples
len(X)

226861

In [19]:
len(X[-1])

100

# Instantiate generative model A with set architecture

In [20]:
model = load_alt_model_a(n_unique=n_unique, T_x=n_time_steps, emb_dim=32, n_a=128)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 32)           1824      
                                                                 
 lstm (LSTM)                 (None, 100, 128)          82432     
                                                                 
 lstm_1 (LSTM)               (None, 128)               131584    
                                                                 
 dense (Dense)               (None, 57)                7353      
                                                                 
 batch_normalization (Batch  (None, 57)                228       
 Normalization)                                                  
                                                                 
 activation (Activation)     (None, 57)                0         
                                                        

# Provide loss, optimizer, and metrics for both alternative models A and B

In [21]:
opt = Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999)
loss = cce_loss()
metrics = [CategoricalAccuracy(), cce_metric()]

# model.compile(loss=loss, optimizer=opt, metrics=metrics)

# Train alternative model A and checkpoint weights

In [22]:
# weights_path = "./weights/weights-improvement-{epoch:02d}-{categorical_accuracy:.4f}.hdf5"
# checkpoint = ModelCheckpoint(weights_path, monitor='categorical_accuracy', verbose=1, save_best_only=True, mode='max')
# callbacks_list = [checkpoint]

In [23]:
# history = model.fit(X, Y, epochs=20, batch_size=2048, callbacks=callbacks_list)

In [24]:
# export_results(history, ['loss'], image_only=False)
# export_results(history, ['categorical_accuracy'], image_only=False)

# Preprocessing for alternative model B

In [25]:
X, Y = init_sequences_b(corpus, char_to_idx, T_x=n_time_steps)
X

<tf.Tensor: shape=(2248, 100), dtype=int64, numpy=
array([[25,  2, 43, ..., 29,  2, 43],
       [29, 40,  2, ..., 36, 39, 43],
       [40, 32, 49, ..., 25, 44,  2],
       ...,
       [39,  2, 30, ..., 32, 33, 43],
       [44, 32, 29, ...,  2, 26, 42],
       [25, 35, 43, ...,  0,  0,  0]], dtype=int64)>

In [26]:
len(X)

2248

In [27]:
len(X[-2])

100

In [28]:
len(X[-1])

100

In [29]:
len(Y[-1])

100

In [30]:
X[-1]

<tf.Tensor: shape=(100,), dtype=int64, numpy=
array([25, 35, 43,  2, 37, 49,  2, 26, 29, 36, 33, 29, 30,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
      dtype=int64)>

In [78]:
Y

<tf.Tensor: shape=(2248, 100), dtype=int64, numpy=
array([[ 2, 43, 33, ...,  2, 43, 44],
       [40,  2, 28, ..., 39, 43, 39],
       [32, 49, 23, ..., 44,  2, 44],
       ...,
       [ 2, 30, 33, ..., 33, 43, 56],
       [32, 29,  2, ..., 26, 42, 29],
       [35, 43,  2, ...,  0,  0,  0]], dtype=int64)>

In [82]:
Y = [tf.one_hot(y, depth=n_unique) for y in tf.reshape(Y, shape=(-1, Y.shape[0]))]

In [83]:
len(Y)

100

# Y here is now a matrix with shape $(T_y, m, n_{unique})$

In [84]:
Y

[<tf.Tensor: shape=(2248, 57), dtype=float32, numpy=
 array([[0., 0., 1., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)>,
 <tf.Tensor: shape=(2248, 57), dtype=float32, numpy=
 array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.]], dtype=float32)>,
 <tf.Tensor: shape=(2248, 57), dtype=float32, numpy=
 array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)>,
 <tf.Tensor: shape=(2248, 57), dtype=float32, 

# Load a saved model
* see arhictecure
* see if prediction will work properly on dummy data

In [31]:
saved_model = load_model('./saved/models/test_model_b.h5')

In [32]:
saved_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 X (InputLayer)              [(None, 50)]                 0         []                            
                                                                                                  
 character-lookup (Embeddin  (None, 50, 32)               1824      ['X[0][0]']                   
 g)                                                                                               
                                                                                                  
 tf.__operators__.getitem (  (None, 32)                   0         ['character-lookup[0][0]']    
 SlicingOpLambda)                                                                                 
                                                                                              

#### Recall that our model needs 3 inputs, X, the hidden state, and the cell state. Because we are generating novel sequences using our trained model we pass in a $(1, 100)$ input where it represents the shape $(m, T_x)$, moreover our hidden and cell states remain the same in terms of their shape which is $(m, n_a)$ but only now it would be $(1, n_a)$ since we are passing only one input example to our model

In [33]:
sample_input = tf.random.uniform(shape=(1, 50), minval=0, maxval=n_unique - 1, dtype=tf.int32)

sample_h = tf.zeros(shape=(1, 128))
sample_c = tf.zeros(shape=(1, 128))

#### use the model to predict an output Y which we know will be of shape $(T_y, m, n_{unique})$ or in this case since we only inputted one example $(T_y, 1, 26)$

In [34]:
saved_model.predict([sample_input, sample_h, sample_c])

1/1 [==============================] - 26s 26s/step


[array([[ 15.871082 , -10.584189 , -40.99686  ,  40.667194 ,  -9.6570835,
          14.49633  , -11.053498 , -18.975246 , -15.9176655, -59.06238  ,
          30.357676 ,  -3.3774421, -22.355906 , -21.894297 ,  -7.28347  ,
         -42.176193 , -19.943626 ,  16.702509 ,  13.931639 ,  35.675907 ,
          14.5267935, -51.186676 ,  -7.1483226,  -0.8430158, -33.155514 ,
          53.309727 , -17.270607 , -15.270151 ,  10.759725 ,   5.5242996,
           8.199804 ,  -5.736205 ,  21.61677  ,  22.939642 ,  -4.2448244,
          16.96171  ,   1.7525232, -20.99272  ,  19.78228  , -30.936378 ,
          10.1721115, -27.408264 ,  33.880375 ,  25.051582 ,   6.1031384,
          12.198984 ,  24.207151 ,  32.50429  , -32.525368 ,   2.9484003,
         -45.78455  ,  -1.2590427,  -4.3932276,  11.26389  ,  -4.111385 ,
         -29.867727 , -25.445957 ]], dtype=float32),
 array([[ 19.611435 , -12.334971 , -40.032116 ,  38.166508 , -11.490919 ,
          16.939375 ,  -7.0738163, -18.87407  , -14.616795 

#### List all layer names
* the goal here is to extract the Embedding, LSTM, Dense, and BatchNormalization layers used in training the model which have all been instantiated once and where we can extract and use as objects in our inference model
* Once we know the layers, we can start by training our model for training, and save the best model with the lowest loss value
* Access the saved models aforementioned layers and use it in the inference model

In [35]:
layers = saved_model.layers
for layer in layers:
    print(layer.name)

X
character-lookup
tf.__operators__.getitem
reshape-layer
init-hidden-state
init-cell-state
tf.__operators__.getitem_1
lstm-cell
tf.__operators__.getitem_2
tf.__operators__.getitem_3
tf.__operators__.getitem_4
tf.__operators__.getitem_5
tf.__operators__.getitem_6
tf.__operators__.getitem_7
tf.__operators__.getitem_8
tf.__operators__.getitem_9
tf.__operators__.getitem_10
tf.__operators__.getitem_11
tf.__operators__.getitem_12
tf.__operators__.getitem_13
tf.__operators__.getitem_14
tf.__operators__.getitem_15
tf.__operators__.getitem_16
tf.__operators__.getitem_17
tf.__operators__.getitem_18
tf.__operators__.getitem_19
tf.__operators__.getitem_20
tf.__operators__.getitem_21
tf.__operators__.getitem_22
tf.__operators__.getitem_23
tf.__operators__.getitem_24
tf.__operators__.getitem_25
tf.__operators__.getitem_26
tf.__operators__.getitem_27
tf.__operators__.getitem_28
tf.__operators__.getitem_29
tf.__operators__.getitem_30
tf.__operators__.getitem_31
tf.__operators__.getitem_32
tf.__operat

In [36]:
lstm_cell = saved_model.get_layer('lstm-cell')
embedding_layer = saved_model.get_layer('character-lookup')
dense_layers = []
norm_layers = []
for layer in layers:
    if "dense" in layer.name:
        dense_layers.append(saved_model.get_layer(layer.name))
    if "norm" in layer.name:
        norm_layers.append(saved_model.get_layer(layer.name))

In [37]:
lstm_cell.get_weights()

[array([[-0.0184154 , -0.02375055,  0.00893003, ..., -0.05158226,
         -0.01664163, -0.21465147],
        [ 0.01582061,  0.08148593,  0.08213706, ..., -0.07070924,
         -0.07361256,  0.09310754],
        [-0.05783164, -0.05902315,  0.02957972, ..., -0.1254873 ,
          0.02566432, -0.00281967],
        ...,
        [ 0.00547936,  0.03153122, -0.03754343, ...,  0.05027683,
         -0.04204738, -0.06839884],
        [-0.14642487,  0.00875235,  0.01889797, ..., -0.0574753 ,
         -0.07182395, -0.01333168],
        [-0.01203713,  0.05460412,  0.02116216, ...,  0.056155  ,
         -0.06590614,  0.07827098]], dtype=float32),
 array([[-0.08455861,  0.03446063,  0.00124661, ...,  0.01920941,
          0.02244429,  0.00455793],
        [-0.00068348, -0.02338182, -0.14515945, ...,  0.01891384,
         -0.00353466,  0.00422234],
        [ 0.03673246, -0.00704461,  0.03165074, ..., -0.04489211,
          0.07992688,  0.12459141],
        ...,
        [ 0.0114749 , -0.00753773, -0.0

In [38]:
embedding_layer.get_weights()

[array([[ 0.02053736, -0.01399976, -0.00965043, ...,  0.0523922 ,
          0.01130588, -0.00432437],
        [-0.04314644,  0.02622067,  0.01322938, ...,  0.06298288,
          0.06555247, -0.01877447],
        [-0.02697898, -0.04954175,  0.00257215, ..., -0.03078171,
         -0.05130325,  0.01310429],
        ...,
        [-0.0806696 ,  0.02140415, -0.02316877, ..., -0.00378218,
         -0.05063841, -0.05605981],
        [-0.05821191, -0.02468988,  0.03918936, ..., -0.00194909,
         -0.01956837,  0.00074366],
        [-0.05939003,  0.03008036,  0.01982103, ...,  0.04486015,
          0.01715827, -0.04844798]], dtype=float32)]

In [39]:
for dense_layer in dense_layers:
    print(dense_layer.get_weights())

[array([[ 0.01212659, -0.04574768, -0.12591755, ...,  0.13249134,
         0.04746192, -0.00502822],
       [-0.01747766, -0.02901762, -0.02991048, ...,  0.1833087 ,
        -0.07888314, -0.13472588],
       [-0.10039048,  0.11795516, -0.04722041, ..., -0.00048793,
         0.03702275,  0.06032871],
       ...,
       [ 0.19415793,  0.15396424,  0.02798049, ...,  0.11210111,
        -0.11689458, -0.01196976],
       [-0.14423203, -0.06492916, -0.07117257, ...,  0.02302504,
         0.03708406, -0.164866  ],
       [-0.16638695, -0.03747589, -0.05845291, ..., -0.15896745,
         0.14201505, -0.10619899]], dtype=float32), array([-2.33176108e-02, -1.77859608e-02, -1.21950600e-02,  5.81323029e-03,
       -2.22110096e-02, -9.50458646e-03,  6.28179312e-03,  9.55670327e-03,
       -2.32631974e-02,  1.83955636e-02, -4.95046377e-02, -3.44706364e-02,
       -3.65040191e-02, -3.04796430e-03,  8.49325396e-03,  1.12325251e-02,
        5.29693207e-03,  2.00526696e-02,  2.69577280e-02,  1.05521167e

In [40]:
for norm_layer in norm_layers:
    print(norm_layer.get_weights())

[array([0.99767464, 0.97929865, 1.0134023 , 0.99949855, 0.9964681 ,
       0.9916438 , 0.998616  , 1.0085002 , 0.9801582 , 1.0340518 ,
       1.0221671 , 1.0576766 , 0.99301445, 0.98694   , 0.9783161 ,
       1.0116867 , 0.99477273, 0.9826209 , 0.99502003, 0.98755777,
       1.0181874 , 1.0187944 , 1.0024788 , 1.001951  , 1.0441748 ,
       1.0024627 , 1.0342499 , 0.9692718 , 1.0112691 , 0.9960474 ,
       0.9722523 , 0.9790469 , 0.9815378 , 1.0136167 , 1.0133615 ,
       0.9944513 , 1.0052651 , 1.006527  , 1.0010744 , 0.9812397 ,
       0.99705136, 1.0231876 , 0.98863804, 1.009697  , 0.96853656,
       1.0054395 , 1.0136415 , 0.9595538 , 1.0028552 , 1.0070016 ,
       0.9772358 , 1.0028921 , 0.99748033, 1.0242583 , 1.0162886 ,
       0.99733156, 0.98232305, 0.9811708 , 0.9722484 , 0.97426826,
       1.0234127 , 1.0020529 , 0.99493355, 1.0058123 ], dtype=float32), array([-0.00806188,  0.00263991, -0.00531885,  0.00236916, -0.01015277,
        0.00096669, -0.00579415,  0.00144756, -0.00

# Load saved weights

In [41]:
saved_model = GenPhiloText(emb_dim=32, n_a=128, n_unique=57, T_x=50, dense_layers_dims=[64, 32, 57])
saved_model([sample_input, sample_h, sample_c])

[<tf.Tensor: shape=(1, 57), dtype=float32, numpy=
 array([[-2.6006547e-03,  2.1261703e-03, -1.6939052e-03, -1.6878630e-03,
          3.9023764e-03, -1.8491967e-03,  2.8346602e-03, -5.9375149e-04,
         -4.0443667e-04, -9.6301443e-04,  1.5775389e-03, -4.4879070e-03,
         -4.0562367e-03,  7.0330949e-05,  2.1970961e-03, -1.2755657e-03,
          4.7378005e-03,  4.5169033e-03, -1.9691600e-03,  2.2887759e-04,
          1.5089827e-03,  6.1523020e-03, -1.2172266e-03, -5.2764476e-04,
         -1.9419387e-03, -3.3924880e-05, -4.2818203e-03,  2.0528613e-03,
          1.9839937e-03,  4.3833810e-03, -2.4296329e-04, -3.2932842e-03,
          8.3094637e-06,  4.5184218e-03,  5.0576301e-03, -6.6638400e-04,
         -1.4995091e-03, -7.1938313e-04,  4.4144010e-03, -1.4213093e-03,
         -9.3885988e-04,  2.4821851e-03,  3.7891831e-04,  5.3404813e-04,
         -2.7426397e-03, -2.8970912e-03,  3.1398414e-03, -7.2672090e-04,
         -3.6944711e-04, -7.8075966e-03, -3.5951310e-03, -3.2252294e-03,
 

In [42]:
saved_model.summary()

Model: "gen_philo_text"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 char-emb-layer (Embedding)  multiple                  1824      
                                                                 
 lstm-cell (LSTM)            multiple                  82432     
                                                                 
 dense-layer-0 (Dense)       multiple                  8256      
                                                                 
 dense-layer-1 (Dense)       multiple                  2080      
                                                                 
 dense-layer-2 (Dense)       multiple                  1881      
                                                                 
 reshape-layer (Reshape)     multiple                  0         
                                                                 
 norm-layer-0 (BatchNormali  multiple               

In [43]:
saved_model.load_weights(filepath='./saved/weights/test_model_gen_philo_text.h5')

In [44]:
saved_model.predict([sample_input, sample_h, sample_c])

1/1 [==============================] - 26s 26s/step


[array([[-4.25585365e+01,  6.68328667e+00, -3.18575878e+01,
         -2.56430912e+01, -2.88827095e+01,  3.77732031e-02,
          5.11136665e+01,  7.37016726e+00,  1.09523916e+01,
          4.72288589e+01, -8.60651076e-01,  4.41161804e+01,
         -1.62293301e+01, -2.71738739e+01,  1.34496117e+01,
         -2.92069912e+01, -5.03112946e+01, -3.38398514e+01,
          8.30157566e+00, -2.18440762e+01,  3.26119194e+01,
          8.60357761e+00, -5.36164856e+00,  8.96211624e+00,
         -2.28650212e+00, -7.46050262e+01, -9.40030212e+01,
          1.27878304e+01,  6.46894169e+00,  6.40227556e+00,
          2.18415947e+01, -4.53465538e+01, -3.48432617e+01,
          1.01727516e+02,  4.83726311e+01, -1.60480289e+01,
          2.58933640e+01,  4.00819168e+01, -7.90776901e+01,
         -1.69180717e+01,  2.50184612e+01, -2.19318867e+01,
         -6.14181089e+00,  2.09584675e+01,  2.64481544e+01,
         -8.54828720e+01, -3.64406776e+01,  1.21836176e+01,
         -3.12865677e+01, -3.12246227e+0

In [45]:
layers = saved_model.layers
for layer in layers:
    print(layer.name)

char-emb-layer
lstm-cell
dense-layer-0
dense-layer-1
dense-layer-2
reshape-layer
norm-layer-0
norm-layer-1


In [46]:
char_emb_layer = saved_model.get_layer('char-emb-layer')
char_emb_layer.get_weights()

[array([[ 0.02111666, -0.00146792,  0.00755065, ...,  0.04362405,
          0.03307302,  0.02691903],
        [ 0.02387214, -0.00062908,  0.04701962, ...,  0.01479399,
          0.02778828,  0.03082614],
        [ 0.02646287,  0.04293448, -0.01753888, ...,  0.04349318,
         -0.0330329 , -0.02963357],
        ...,
        [ 0.0218588 , -0.01277796,  0.05689206, ...,  0.0344196 ,
         -0.03902251,  0.00024158],
        [ 0.00710455, -0.05463982,  0.05671924, ..., -0.05709765,
          0.03724542, -0.0207356 ],
        [ 0.02407178, -0.00727064,  0.01547497, ...,  0.06202401,
         -0.03386848, -0.02553537]], dtype=float32)]

In [47]:
lstm_cell = saved_model.get_layer('lstm-cell')
lstm_cell.get_weights()

[array([[-0.06620786,  0.06320175,  0.12758836, ...,  0.03354447,
         -0.0471941 ,  0.06856976],
        [-0.02823958, -0.0211705 , -0.0937408 , ..., -0.08329665,
          0.02793497, -0.02530165],
        [ 0.0220161 , -0.0879951 , -0.07927505, ...,  0.07446331,
         -0.00644632, -0.01354439],
        ...,
        [ 0.02759714, -0.06205938,  0.09160436, ..., -0.07842141,
          0.08939886, -0.01395737],
        [-0.10843082, -0.10630415,  0.10906404, ...,  0.0093196 ,
          0.04554187, -0.01853471],
        [ 0.05218415, -0.06030306, -0.11617827, ..., -0.04195077,
          0.06512791,  0.0812979 ]], dtype=float32),
 array([[-4.12605442e-02, -2.10973322e-02, -7.91592337e-03, ...,
         -7.46237859e-02,  3.37877721e-02, -1.23985577e-04],
        [-2.00786531e-01, -5.60098663e-02,  8.44157040e-02, ...,
         -1.03415564e-01, -7.92377442e-02,  1.94198295e-01],
        [-1.60260703e-02, -4.79991101e-02, -6.18406869e-02, ...,
          5.47376275e-02, -4.24294136e-02

In [48]:
dense_layers = []
norm_layers = []
for layer in layers:
    if "dense" in layer.name:
        dense_layers.append(saved_model.get_layer(layer.name))
    if "norm" in layer.name:
        norm_layers.append(saved_model.get_layer(layer.name))

In [49]:
for dense_layer in dense_layers:
    print(dense_layer.get_weights())

[array([[ 0.04901963, -0.17243719,  0.03184679, ...,  0.17619476,
         0.09527965, -0.02295073],
       [-0.00939342,  0.14603478,  0.00354737, ..., -0.05026373,
         0.10742786,  0.12851414],
       [-0.0397801 ,  0.02911105,  0.14048885, ..., -0.05871448,
         0.04467083, -0.0549162 ],
       ...,
       [-0.06237885,  0.06461646, -0.05215921, ..., -0.05943538,
         0.10453384,  0.1755654 ],
       [ 0.10375043, -0.15292254,  0.02860362, ...,  0.08474719,
        -0.00586738, -0.16241167],
       [-0.05342362,  0.01044544,  0.12475466, ...,  0.14673452,
        -0.15581708,  0.13595714]], dtype=float32), array([-2.80400012e-02, -3.50865861e-03,  1.89379361e-02,  1.41829427e-04,
        3.97135951e-02, -2.30618734e-02, -9.11550233e-05, -2.84410687e-03,
       -5.87508373e-04, -8.57352000e-03, -2.66288477e-03,  1.72079261e-02,
       -4.10088748e-02,  4.18956913e-02, -6.70768972e-03,  3.23303826e-02,
       -5.08050397e-02, -2.63574142e-02, -1.01870168e-02,  2.54463032e

In [50]:
for norm_layer in norm_layers:
    print(norm_layer.get_weights())

[array([0.9711099 , 1.0090858 , 1.0131179 , 0.9911795 , 0.97761554,
       1.0306649 , 0.9613138 , 0.9998002 , 0.9920044 , 1.0002866 ,
       1.0228332 , 1.0159761 , 1.0123972 , 1.020887  , 0.97525865,
       0.98493063, 0.98295224, 1.0128604 , 0.9860124 , 0.9982375 ,
       0.98625934, 0.9705311 , 1.0309348 , 1.0319668 , 0.9934609 ,
       1.0002123 , 0.99883854, 1.0205432 , 0.9869614 , 1.0703921 ,
       0.95673156, 1.0057505 , 0.99894375, 1.0266067 , 0.99101174,
       0.96853167, 0.9797536 , 0.9968553 , 1.0144676 , 1.0278354 ,
       0.98036474, 0.9665136 , 0.94981813, 0.9814051 , 0.9968354 ,
       1.0195315 , 0.9733389 , 1.0408456 , 1.0193908 , 0.98108965,
       1.0251061 , 0.9830418 , 1.0017284 , 0.9901122 , 1.0000855 ,
       0.97785383, 0.99806255, 1.0224919 , 1.0144665 , 1.0111854 ,
       0.9734452 , 1.0043703 , 1.0046436 , 1.0074971 ], dtype=float32), array([ 0.00272677, -0.0058729 , -0.00302456, -0.00957917, -0.00817027,
       -0.00411418, -0.00945891,  0.0025301 , -0.01

In [51]:
inference_model = load_inf_model(char_emb_layer, lstm_cell, dense_layers, norm_layers, char_to_idx)

57
128
tf.Tensor(
[[-inf   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.]], shape=(1, 57), dtype=float32)
100
KerasTensor(type_spec=TensorSpec(shape=(None, 1, 32), dtype=tf.float32, name=None), name='char-emb-layer/embedding_lookup/Identity:0', description="created by layer 'char-emb-layer'")
0
KerasTensor(type_spec=TensorSpec(shape=(None, 1, 32), dtype=tf.float32, name=None), name='char-emb-layer/embedding_lookup/Identity:0', description="created by layer 'char-emb-layer'")
1
KerasTensor(type_spec=TensorSpec(shape=(None, 1, 32), dtype=tf.float32, name=None), name='char-emb-layer/embedding_lookup/Identity:0', description="created by layer 'char-emb-layer'")
2
KerasTensor(type_spec=TensorSpec(shape=(None, 1, 32), dtype=tf.float32, name=None), name='ch

In [52]:
inference_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 1)]                  0         []                            
                                                                                                  
 char-emb-layer (Embedding)  (None, 1, 32)                1824      ['input_2[0][0]',             
                                                                     'reshape[0][0]',             
                                                                     'reshape[1][0]',             
                                                                     'reshape[2][0]',             
                                                                     'reshape[3][0]',             
                                                                     'reshape[4][0]',         

In [73]:
sample_input = tf.random.uniform(shape=(1, 1), minval=0, maxval=n_unique - 1, dtype=tf.int32)

sample_h = tf.zeros(shape=(1, 128))
sample_c = tf.zeros(shape=(1, 128))

In [74]:
sample_input

<tf.Tensor: shape=(1, 1), dtype=int32, numpy=array([[17]])>

In [75]:
pred_ids = inference_model.predict([sample_input, sample_h, sample_c])

1/1 [==============================] - 0s 52ms/step


In [76]:
pred_ids

[array([[33]], dtype=int64),
 array([[33]], dtype=int64),
 array([[33]], dtype=int64),
 array([[33]], dtype=int64),
 array([[33]], dtype=int64),
 array([[50]], dtype=int64),
 array([[50]], dtype=int64),
 array([[50]], dtype=int64),
 array([[50]], dtype=int64),
 array([[50]], dtype=int64),
 array([[50]], dtype=int64),
 array([[50]], dtype=int64),
 array([[14]], dtype=int64),
 array([[14]], dtype=int64),
 array([[14]], dtype=int64),
 array([[14]], dtype=int64),
 array([[14]], dtype=int64),
 array([[14]], dtype=int64),
 array([[14]], dtype=int64),
 array([[14]], dtype=int64),
 array([[14]], dtype=int64),
 array([[14]], dtype=int64),
 array([[14]], dtype=int64),
 array([[25]], dtype=int64),
 array([[25]], dtype=int64),
 array([[25]], dtype=int64),
 array([[10]], dtype=int64),
 array([[25]], dtype=int64),
 array([[25]], dtype=int64),
 array([[25]], dtype=int64),
 array([[25]], dtype=int64),
 array([[25]], dtype=int64),
 array([[25]], dtype=int64),
 array([[25]], dtype=int64),
 array([[38]],

In [77]:
decode_predictions(pred_ids, idx_to_char)

'iiiiizzzzzzz11111111111aaa-aaaaaaannuuuuuuuu&&&&&&&&&&&ggggggggggggggggggggggggggggggggggwwwwwwwwaaa'